# Objective
Forecast sales (no. of items sold) for each item and store for the next 28 days

20200527: Diff from m5-forecasting_v1.ipynb: Here we do predictions for 1 state at a time

20200606: Diff from m5-forecasting_v2.ipynb: Here we do more feature engineering e.g. target_store, target_item, target_dept

In [208]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import gc
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time

from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor

%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [209]:
##### Input params ########
state = 'CA'
store = 'CA_1' # CA_1, CA_2, CA_3, CA_4, TX_1, TX_2, TX_3, WI_1, WI_2, WI_3

model_seed = 102
sample_seed = 103              # for sampling the dataframe

n_estimators = 100             # for the initial model before tuning. default = 100
max_depth = 3                  # for the initial model before tuning. default = 3
learning_rate = 0.1            # for the initial model before tuning. default = 0.1
min_child_weight = 1           # for the initial model before tuning. default = 1
subsample = 1                  # for the initial model before tuning. default = 1
colsample_bytree = 1           # for the initial model before tuning. default = 1
colsample_bylevel = 1          # for the initial model before tuning. default = 1
train_test_split_seed = 111    # 111
model_seed = 100

train_start = 0
train_end = 1913               # use date_block_num from train_start to train_end as train set
val_date_block_num = 1913      # date_block_num for validation set
# pred_start = 1886              # do pred starting from this day
# pred_end = 1913                # do pred until and include this day
pred_start = 1914            # do pred starting from this day
pred_end = 1941              # do pred until and include this day

shift_range = [1, 2, 3, 4, 5, 6, 7, 30, 365] # Use values from last 7 days, from 30 days ago, and from 365 days ago

fontsize = 14
ticklabelsize = 14
###########################

In [210]:
ticStart = time.time()

# Common functions

In [211]:
def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int16`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int16)
    
    return df

def get_lags(index_cols, cols_to_rename, all_data, shift_range):
    """
    For a dataframe all_data, get the lags specified in cols_to_rename
    Input
        index_cols    : e.g. ['item_id', 'store_id', 'date_block_num']. These are the cols to do merging with the
                        lagged dataset
        cols_to_rename: e.g. ['target']. Based on this example, all_data will be returned with columns target_lag_1, 
                        target_lag_2, ..., target_lag_365.
        all_data      : need to have a column 'date_block_num' to indicate chronological order
        shift_range   : e.g. [1, 2, 3, 4, 5, 6, 7, 30, 365]
    Output
        all_data
    """
    for day_shift in shift_range:
        train_shift = all_data[index_cols + cols_to_rename].copy()
    
        # E.g. variable of 0 becomes 1, for day_shift = 1.
        # So when this is merged with variable of 1 in all_data, this will represent lag of 1.
        train_shift['date_block_num'] = train_shift['date_block_num'] + day_shift
    
        foo = lambda x: '{}_lag_{}'.format(x, day_shift) if x in cols_to_rename else x
        train_shift = train_shift.rename(columns=foo)

        all_data = pd.merge(all_data, train_shift, on=index_cols, how='left')
    
        # Fill the NaNs with 0
        for x in cols_to_rename:
            all_data[x + '_lag_' + str(day_shift)].fillna(0, inplace=True)
    
    del train_shift
    gc.collect();

    return all_data


In [212]:
def get_agg_features(all_data):
    """
    Do feature engineering to get 3 features for all_data: target_store, target_item, target_dept
    """
    # Create (shop_id, date_block_num) aggregates
    # This gives the number of items each shop sold per date_block_num as a feature
    # We won't have the above for the test set, so can only use its lagged features 
    gb = all_data.groupby(['store_id', 'date_block_num'], as_index=False).agg({'target':[('target_store','sum')]})
    gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
    all_data = all_data.merge(gb, 
                              left_on=['store_id', 'date_block_num'], 
                              right_on=['store_id', 'date_block_num'],
                              how='left').fillna(0)
    
    # Create (item_id, date_block_num) aggregates
    # This gives the number of items sold per item_id per date_block_num as a feature
    # We won't have the above for the test set, so can only use its lagged features 
    gb = all_data.groupby(['item_id', 'date_block_num'], as_index=False).agg({'target':[('target_item','sum')]})
    gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
    all_data = all_data.merge(gb, 
                              left_on=['item_id', 'date_block_num'], 
                              right_on=['item_id', 'date_block_num'],
                              how='left').fillna(0)
    
    # Create (dept_id, date_block_num) aggregates
    # This gives the number of items sold per dept_id per date_block_num as a feature
    # We won't have the above for the test set, so can only use its lagged features 
    gb = all_data.groupby(['dept_id', 'date_block_num'], as_index=False).agg({'target':[('target_dept','sum')]})
    gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
    all_data = all_data.merge(gb, 
                              left_on=['dept_id', 'date_block_num'], 
                              right_on=['dept_id', 'date_block_num'],
                              how='left').fillna(0)
    
    return all_data

# Load data

In [213]:
sell_prices = pd.read_csv("./data/sell_prices.csv")
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


File 2: “sell_prices.csv”

Contains information about the price of the products sold per store and date.
* store_id: The id of the store where the product is sold. 
* item_id: The id of the product.
* wm_yr_wk: The id of the week.
* sell_price: The price of the product for the given week/store. The price is provided per week (average across seven days). If not available, this means that the product was not sold during the examined week. Note that although prices are constant at weekly basis, they may change through time (both training and test set).

In [214]:
calendar = pd.read_csv("./data/calendar.csv")
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,1,1,1
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,1,0,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,1,1,1
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,1,1,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,1,1,0


File 1: “calendar.csv” 

Contains information about the dates the products are sold.
* date: The date in a “y-m-d” format.
* wm_yr_wk: The id of the week the date belongs to.
* weekday: The type of the day (Saturday, Sunday, …, Friday).
* wday: The id of the weekday, starting from Saturday.
* month: The month of the date.
* year: The year of the date.
* event_name_1: If the date includes an event, the name of this event.
* event_type_1: If the date includes an event, the type of this event.
* event_name_2: If the date includes a second event, the name of this event.
* event_type_2: If the date includes a second event, the type of this event.
* snap_CA, snap_TX, and snap_WI: A binary variable (0 or 1) indicating whether the stores of CA, TX or WI allow SNAP purchases on the examined date. 1 indicates that SNAP purchases are allowed.


In [215]:
sales_tr_val = pd.read_csv("./data/sales_train_validation.csv")
print("len(sales_tr_val) = " + str(len(sales_tr_val)))
sales_tr_val.head()

len(sales_tr_val) = 30490


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


For each (item_id, store_id), we have the number of items sold for 1913 days or 5.24 years.

There are 30,490 distinct id's, and 1913 days which gives 30,490*1913 = 58,327,370

In [216]:
# Keep only data from one state
sales_tr_val = sales_tr_val[(sales_tr_val['state_id']==state) & (sales_tr_val['store_id']==store)]
print("len(sales_tr_val) = " + str(len(sales_tr_val)))

len(sales_tr_val) = 3049


In [217]:
sample_sub = pd.read_csv("./data/sample_submission.csv")
print(len(sample_sub))
sample_sub.head()

60980


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Provide the forecast for each id for the next 28 days. For the validation rows, this corresponds to d_1914 - d_1941, and for the evaluation rows, this corresponds to d_1942 - d_1969. (Note: a month before the competition close, the ground truth for the validation rows will be provided.)

Note 30,490*2 = 60,980 which is the number of rows in the submission file.

# Pre-process data

In [218]:
print(calendar.memory_usage().sum())

# # In calendar 'd' column, change 'd_1' to 1
# calendar['d'] = calendar.apply(lambda row: row['d'].split('_')[1], axis=1)

# Replance NaN with nil
calendar = calendar.fillna(value = 'nil')

# Downcast dtypes from 64 bit to save memory
calendar= downcast_dtypes(calendar)

# Change to category dtype
calendar[["year", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]] = \
    calendar[["year", "event_name_1", "event_type_1", "event_name_2", "event_type_2"]] .astype("category")

# Drop weekday column since its not used
calendar.drop(['weekday'], axis=1, inplace=True)

print(calendar.memory_usage().sum())

calendar.head()

220608
67297


,date,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,1,1,2011,d_1,nil,nil,nil,nil,0,0,0
1,2011-01-30,11101,2,1,2011,d_2,nil,nil,nil,nil,0,0,0
2,2011-01-31,11101,3,1,2011,d_3,nil,nil,nil,nil,0,0,0
3,2011-02-01,11101,4,2,2011,d_4,nil,nil,nil,nil,1,1,0
4,2011-02-02,11101,5,2,2011,d_5,nil,nil,nil,nil,1,0,1


In [219]:
# Downcast dtypes from 64 bit to save memory
print(sell_prices.memory_usage().sum())
sell_prices = downcast_dtypes(sell_prices)

sell_prices[["store_id", "item_id"]] = \
    sell_prices[["store_id","item_id"]].astype("category")

print(sell_prices.memory_usage().sum())

218915952
61676881


In [220]:
# Downcast dtypes from 64 bit to save memory
print(sales_tr_val.memory_usage().sum())
sales_tr_val = downcast_dtypes(sales_tr_val)

sales_tr_val[["item_id", "dept_id", "cat_id", "store_id", "state_id"]] = \
    sales_tr_val[["item_id", "dept_id", "cat_id", "store_id", "state_id"]].astype("category")

print(sales_tr_val.memory_usage().sum())

46832640
11839520


In [221]:
# Melt sales_tr_val to switch from a wide to a long dataframe
tic = time.time()
value_vars = ['d_' + str(x) for x in range(1, 1914)]
sale_tr_val_melt = pd.melt(sales_tr_val, 
                           id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], 
                           value_vars=value_vars)

toc = time.time()
print("Time taken = " + str(toc-tic) + ' s')
sale_tr_val_melt.head()

Time taken = 12.445109128952026 s


,id,item_id,dept_id,cat_id,store_id,state_id,variable,value
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0


In [222]:
del sales_tr_val
gc.collect();
sales_tr_val = pd.DataFrame()

In [223]:
# Merge with calendar to get events info
sale_tr_val_melt_merged = sale_tr_val_melt.merge(calendar, 
                                                 left_on='variable', 
                                                 right_on='d', 
                                                 how='left')

# Drop unneccesary columns
sale_tr_val_melt_merged.drop(columns=['d'], inplace=True)

sale_tr_val_melt_merged.head()

,id,item_id,dept_id,cat_id,store_id,state_id,variable,value,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,1,2011,nil,nil,nil,nil,0,0,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,1,2011,nil,nil,nil,nil,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,1,2011,nil,nil,nil,nil,0,0,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,1,2011,nil,nil,nil,nil,0,0,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,1,1,2011,nil,nil,nil,nil,0,0,0


In [224]:
# del calendar
# gc.collect();
# calendar = pd.DataFrame()

# del sale_tr_val_melt
# gc.collect();
# sale_tr_val_melt = pd.DataFrame()

In [225]:
# Merge with sell_prices to get price info
sale_tr_val_melt_merged2 = sale_tr_val_melt_merged.merge(sell_prices, 
                                                         left_on=['store_id', 'item_id', 'wm_yr_wk'], 
                                                         right_on=['store_id', 'item_id', 'wm_yr_wk'], 
                                                         how='left')
sale_tr_val_melt_merged2.head()

,id,item_id,dept_id,cat_id,store_id,state_id,variable,value,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN


In [226]:
# del sell_prices
# gc.collect();
# sell_prices = pd.DataFrame()

del sale_tr_val_melt_merged
gc.collect();
sale_tr_val_melt_merged = pd.DataFrame()

In [227]:
# Get no. of distinct values in each column
print("item_id uniques = " + str(sale_tr_val_melt_merged2['item_id'].nunique()))
print("dept_id uniques = " + str(sale_tr_val_melt_merged2['dept_id'].nunique()))
print("cat_id uniques = " + str(sale_tr_val_melt_merged2['cat_id'].nunique()))
print("store_id uniques = " + str(sale_tr_val_melt_merged2['store_id'].nunique()))
print("state_id uniques = " + str(sale_tr_val_melt_merged2['state_id'].nunique()))

item_id uniques = 3049
dept_id uniques = 7
cat_id uniques = 3
store_id uniques = 1
state_id uniques = 1


In [228]:
print(sale_tr_val_melt_merged2['dept_id'].unique())

[HOBBIES_1, HOBBIES_2, HOUSEHOLD_1, HOUSEHOLD_2, FOODS_1, FOODS_2, FOODS_3]
Categories (7, object): [HOBBIES_1, HOBBIES_2, HOUSEHOLD_1, HOUSEHOLD_2, FOODS_1, FOODS_2, FOODS_3]


In [229]:
print(sale_tr_val_melt_merged2['cat_id'].unique())

[HOBBIES, HOUSEHOLD, FOODS]
Categories (3, object): [HOBBIES, HOUSEHOLD, FOODS]


In [230]:
print(sale_tr_val_melt_merged2['store_id'].unique())

['CA_1']


In [231]:
print(sale_tr_val_melt_merged2['state_id'].unique())

[CA]
Categories (1, object): [CA]


In [232]:
# Convert variable to a numerical column ie. d_1 to 1
sale_tr_val_melt_merged2['variable'] = sale_tr_val_melt_merged2['variable'].str.split('_').str[1]
sale_tr_val_melt_merged2

,id,item_id,dept_id,cat_id,store_id,state_id,variable,value,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,NaN
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,1,12,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,0.46
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,1,2,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,1.56
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,1,0,2011-01-29,11101,...,1,2011,nil,nil,nil,nil,0,0,0,3.17


In [233]:
# Convert variable to int
sale_tr_val_melt_merged2['variable'] = sale_tr_val_melt_merged2['variable'].astype('int16')

# Create features

In [234]:
all_data = sale_tr_val_melt_merged2
all_data = all_data.rename(columns={'variable': 'date_block_num', 
                                    'value': 'target'})

In [235]:
# Remove unnecessaray column
all_data.drop(['date'], axis=1, inplace=True)

In [236]:
# Convert features to categorical
columns_to_be_labeled = [
    'year',
    'event_name_1',
    'event_type_1',
    'event_name_2',
    'event_type_2',
    'wm_yr_wk',
    'item_id',
    'store_id',
    'dept_id',
    'cat_id',
    'state_id'
]
for column in columns_to_be_labeled:
    all_data.loc[:, column] = LabelEncoder().fit_transform(all_data[column]) # fit_transform: Fit label encoder and return encoded labels. Note the encoded labels are integers!!

all_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date_block_num,target,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,0,0,1,1,0,30,4,4,2,0,0,0,NaN
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,1,0,0,1,1,0,30,4,4,2,0,0,0,NaN
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,1,0,0,1,1,0,30,4,4,2,0,0,0,NaN
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1,0,0,1,1,0,30,4,4,2,0,0,0,NaN
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,1,0,0,1,1,0,30,4,4,2,0,0,0,NaN


In [237]:
# Create (shop_id, date_block_num) aggregates
# This gives the number of items each shop sold per date_block_num as a feature
# We won't have the above for the test set, so can only use its lagged features 
gb = all_data.groupby(['store_id', 'date_block_num'], as_index=False).agg({'target':{'target_store':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = all_data.merge(gb, 
                          left_on=['store_id', 'date_block_num'], 
                          right_on=['store_id', 'date_block_num'],
                          how='left').fillna(0)
all_data.head()

/Users/yibin/anaconda3/lib/python3.7/site-packages/pandas/core/groupby/generic.py:1315: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


,id,item_id,dept_id,cat_id,store_id,state_id,date_block_num,target,wm_yr_wk,wday,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,target_store
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,0,0,1,...,0,30,4,4,2,0,0,0,0.0,4337
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,1,0,0,1,...,0,30,4,4,2,0,0,0,0.0,4337
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,1,0,0,1,...,0,30,4,4,2,0,0,0,0.0,4337
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1,0,0,1,...,0,30,4,4,2,0,0,0,0.0,4337
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,1,0,0,1,...,0,30,4,4,2,0,0,0,0.0,4337


In [238]:
# Create (item_id, date_block_num) aggregates
# This gives the number of items sold per item_id per date_block_num as a feature
# We won't have the above for the test set, so can only use its lagged features 
gb = all_data.groupby(['item_id', 'date_block_num'], as_index=False).agg({'target':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = all_data.merge(gb, 
                          left_on=['item_id', 'date_block_num'], 
                          right_on=['item_id', 'date_block_num'],
                          how='left').fillna(0)
all_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date_block_num,target,wm_yr_wk,wday,...,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,target_store,target_item
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,0,0,1,...,30,4,4,2,0,0,0,0.0,4337,0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,1,0,0,1,...,30,4,4,2,0,0,0,0.0,4337,0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,1,0,0,1,...,30,4,4,2,0,0,0,0.0,4337,0
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1,0,0,1,...,30,4,4,2,0,0,0,0.0,4337,0
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,1,0,0,1,...,30,4,4,2,0,0,0,0.0,4337,0


In [239]:
# Create (dept_id, date_block_num) aggregates
# This gives the number of items sold per dept_id per date_block_num as a feature
# We won't have the above for the test set, so can only use its lagged features 
gb = all_data.groupby(['dept_id', 'date_block_num'], as_index=False).agg({'target':{'target_dept':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = all_data.merge(gb, 
                          left_on=['dept_id', 'date_block_num'], 
                          right_on=['dept_id', 'date_block_num'],
                          how='left').fillna(0)
all_data.head()

,id,item_id,dept_id,cat_id,store_id,state_id,date_block_num,target,wm_yr_wk,wday,...,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,target_store,target_item,target_dept
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,0,0,1,...,4,4,2,0,0,0,0.0,4337,0,528
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,1,0,0,1,...,4,4,2,0,0,0,0.0,4337,0,528
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,1,0,0,1,...,4,4,2,0,0,0,0.0,4337,0,528
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1,0,0,1,...,4,4,2,0,0,0,0.0,4337,0,528
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,1,0,0,1,...,4,4,2,0,0,0,0.0,4337,0,528


In [240]:
# Create lag features
index_cols = ['item_id', 'store_id', 'date_block_num']
cols_to_rename = ['target', 'target_store', 'target_item', 'target_dept']

for day_shift in shift_range:
    print(day_shift)
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    # E.g. variable of 0 becomes 1, for day_shift = 1.
    # So when this is merged with variable of 1 in all_data, this will represent lag of 1.
    train_shift['date_block_num'] = train_shift['date_block_num'] + day_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, day_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left')
    
    # Fill the NaNs with 0
    for x in cols_to_rename:
        all_data[x + '_lag_' + str(day_shift)].fillna(0, inplace=True)
    
del train_shift
gc.collect();

all_data

1
2
3
4
5
6
7
30
365


,id,item_id,dept_id,cat_id,store_id,state_id,date_block_num,target,wm_yr_wk,wday,...,target_item_lag_7,target_dept_lag_7,target_lag_30,target_store_lag_30,target_item_lag_30,target_dept_lag_30,target_lag_365,target_store_lag_365,target_item_lag_365,target_dept_lag_365
0,HOBBIES_1_001_CA_1_validation,1437,3,1,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HOBBIES_1_002_CA_1_validation,1438,3,1,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,1439,3,1,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HOBBIES_1_004_CA_1_validation,1440,3,1,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,HOBBIES_1_005_CA_1_validation,1441,3,1,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,HOBBIES_1_006_CA_1_validation,1442,3,1,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,HOBBIES_1_007_CA_1_validation,1443,3,1,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,HOBBIES_1_008_CA_1_validation,1444,3,1,0,0,1,12,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,HOBBIES_1_009_CA_1_validation,1445,3,1,0,0,1,2,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,HOBBIES_1_010_CA_1_validation,1446,3,1,0,0,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [241]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832737 entries, 0 to 5832736
Data columns (total 59 columns):
id                      object
item_id                 int64
dept_id                 int64
cat_id                  int64
store_id                int64
state_id                int64
date_block_num          int16
target                  int16
wm_yr_wk                int64
wday                    int16
month                   int16
year                    int64
event_name_1            int64
event_type_1            int64
event_name_2            int64
event_type_2            int64
snap_CA                 int16
snap_TX                 int16
snap_WI                 int16
sell_price              float32
target_store            int16
target_item             int16
target_dept             int16
target_lag_1            float64
target_store_lag_1      float64
target_item_lag_1       float64
target_dept_lag_1       float64
target_lag_2            float64
target_store_lag_2      float64
ta

# Mean encoding

In [242]:
# !!!! The below is not implemented correcly !!!!

# kf = KFold(n_splits=2, shuffle=False)

# for tr_ind, val_ind in kf.split(all_data):
#     X_tr, X_val = all_data.iloc[tr_ind], all_data.iloc[val_ind]
    
#     # Calculate a mapping: {item_id: target_mean}
#     item_id_target_mean = X_tr[(all_data['date_block_num']>=train_end-30) & 
#                                (all_data['date_block_num']<=train_end)].groupby('item_id').target.mean()

#     # Map the computed means to the `item_id`'s
#     all_data.loc[all_data.index[val_ind], 'item_target_enc'] = X_val['item_id'].map(item_id_target_mean)
    
# # # Fill NaNs
# # all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# all_data

# Split into train, val, test sets

We use 30 days as validation data

In [243]:
to_drop_cols = ['id', 'target', 'date_block_num', 'state_id', 'store_id',
                'target_store', 'target_item', 'target_dept']

In [244]:
X_train = all_data[(all_data['date_block_num']>=train_start) & 
                   (all_data['date_block_num']<=train_end)].drop(to_drop_cols, axis = 1)
X_cv = all_data[all_data['date_block_num']==val_date_block_num].drop(to_drop_cols, axis = 1)
# X_train_cv = all_data[all_data['date_block_num']<=1913].drop(to_drop_cols, axis = 1)

y_train = all_data[(all_data['date_block_num']>=train_start) & 
                   (all_data['date_block_num']<=train_end)][['id', 'target']]
y_cv = all_data[all_data['date_block_num']==val_date_block_num][['id', 'target']]
# y_train_cv = all_data[all_data['date_block_num']<=1913][['id', 'target']]

# Train the model and get prediction for day val_date_block_num

In [245]:
# Create the model
model = XGBRegressor(seed=model_seed,
                      n_estimators=n_estimators,
                      max_depth=max_depth,
                      learning_rate=learning_rate,
                      min_child_weight=min_child_weight,
                      objective='reg:squarederror')

# Train the regressor
model.fit(X_train, y_train['target'])

/Users/yibin/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/Users/yibin/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[18:09:31] WARNING: src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=100, silent=None, subsample=1, verbosity=1)

In [246]:
# View a list of the features and their importance scores
imp = list(zip(X_train.columns, model.feature_importances_))
imp.sort(key=lambda tup: tup[1], reverse=False) 
imp

[('dept_id', 0.0),
 ('cat_id', 0.0),
 ('year', 0.0),
 ('event_name_2', 0.0),
 ('event_type_2', 0.0),
 ('snap_TX', 0.0),
 ('snap_WI', 0.0),
 ('target_item_lag_1', 0.0),
 ('target_item_lag_2', 0.0),
 ('target_item_lag_3', 0.0),
 ('target_dept_lag_3', 0.0),
 ('target_item_lag_4', 0.0),
 ('target_store_lag_5', 0.0),
 ('target_item_lag_5', 0.0),
 ('target_dept_lag_5', 0.0),
 ('target_item_lag_6', 0.0),
 ('target_item_lag_7', 0.0),
 ('target_store_lag_30', 0.0),
 ('target_item_lag_30', 0.0),
 ('target_dept_lag_30', 0.0),
 ('target_store_lag_365', 0.0),
 ('target_item_lag_365', 0.0),
 ('target_dept_lag_365', 0.0),
 ('target_store_lag_4', 0.0004630705),
 ('event_type_1', 0.00048953935),
 ('target_dept_lag_4', 0.0012799372),
 ('snap_CA', 0.0013667896),
 ('event_name_1', 0.001701893),
 ('target_dept_lag_6', 0.0017422244),
 ('target_dept_lag_1', 0.0018121789),
 ('wm_yr_wk', 0.0029294507),
 ('target_store_lag_6', 0.003199476),
 ('target_store_lag_2', 0.0033363407),
 ('target_dept_lag_7', 0.0033445

In [247]:
# Do prediction
pred = model.predict(X_cv).astype('int')

# Calculate RMSE
print("MSE = " + str(mean_squared_error(y_cv['target'], pred)))

MSE = 5.38602820596917


In [248]:
y_cv['pred'] = pred
y_cv

,id,target,pred
5829688,HOBBIES_1_001_CA_1_validation,1,1
5829689,HOBBIES_1_002_CA_1_validation,0,0
5829690,HOBBIES_1_003_CA_1_validation,1,1
5829691,HOBBIES_1_004_CA_1_validation,2,3
5829692,HOBBIES_1_005_CA_1_validation,4,1
5829693,HOBBIES_1_006_CA_1_validation,0,0
5829694,HOBBIES_1_007_CA_1_validation,1,0
5829695,HOBBIES_1_008_CA_1_validation,1,4
5829696,HOBBIES_1_009_CA_1_validation,0,0
5829697,HOBBIES_1_010_CA_1_validation,2,0


# Get predictions for validation set 28 days

In [249]:
# Rename columns
sale_tr_val_melt = sale_tr_val_melt.rename(columns={'variable': 'date_block_num', 
                                                    'value': 'target'})

In [250]:
# Convert variable to a numerical column ie. d_1 to 1
sale_tr_val_melt['date_block_num'] = sale_tr_val_melt['date_block_num'].str.split('_').str[1].astype('int16')
sale_tr_val_melt.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,date_block_num,target
5832732,FOODS_3_823_CA_1_validation,FOODS_3_823,FOODS_3,FOODS,CA_1,CA,1913,1
5832733,FOODS_3_824_CA_1_validation,FOODS_3_824,FOODS_3,FOODS,CA_1,CA,1913,0
5832734,FOODS_3_825_CA_1_validation,FOODS_3_825,FOODS_3,FOODS,CA_1,CA,1913,2
5832735,FOODS_3_826_CA_1_validation,FOODS_3_826,FOODS_3,FOODS,CA_1,CA,1913,0
5832736,FOODS_3_827_CA_1_validation,FOODS_3_827,FOODS_3,FOODS,CA_1,CA,1913,21


In [251]:
# Convert variable to a numerical column ie. d_1 to 1
calendar['d'] = calendar['d'].str.split('_').str[1].astype('int16')
calendar.head()

,date,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,1,1,2011,1,nil,nil,nil,nil,0,0,0
1,2011-01-30,11101,2,1,2011,2,nil,nil,nil,nil,0,0,0
2,2011-01-31,11101,3,1,2011,3,nil,nil,nil,nil,0,0,0
3,2011-02-01,11101,4,2,2011,4,nil,nil,nil,nil,1,1,0
4,2011-02-02,11101,5,2,2011,5,nil,nil,nil,nil,1,0,1


In [252]:
# Do prediction on the test set
results = y_cv[['id']].copy()
sale_tr_val_melt2 = sale_tr_val_melt[sale_tr_val_melt['date_block_num']<pred_start]
mse = []

for i in range(pred_start, pred_end+1):
    print("\nday = " + str(i))
    
    # Create the test set here
    temp = sale_tr_val_melt2[sale_tr_val_melt2['date_block_num']==1].copy()
    temp.loc[:, 'target'] = 0
    temp.loc[:, 'date_block_num'] = i
    sale_tr_val_melt2 = sale_tr_val_melt2.append(temp)
    
    # Merge with calendar to get events info
    sale_tr_val_melt_merged = sale_tr_val_melt2.merge(calendar, 
                                                     left_on='date_block_num', 
                                                     right_on='d', 
                                                     how='left')

    # Drop unneccesary columns
    sale_tr_val_melt_merged.drop(columns=['d'], inplace=True)
    
    # Merge with sell_prices to get price info
    all_data = sale_tr_val_melt_merged.merge(sell_prices, 
                                             left_on=['store_id', 'item_id', 'wm_yr_wk'], 
                                             right_on=['store_id', 'item_id', 'wm_yr_wk'], 
                                             how='left')

    # Remove unnecessaray column
    all_data.drop(['date'], axis=1, inplace=True)
    
    # Convert features to categorical 
    for column in columns_to_be_labeled:
        all_data.loc[:, column] = LabelEncoder().fit_transform(all_data[column]) # fit_transform: Fit label encoder and return encoded labels. Note the encoded labels are integers!!

    # Create features
    all_data = get_agg_features(all_data)
       
    # Get the lag features
    all_data = get_lags(index_cols, cols_to_rename, all_data, shift_range)
    
    # Get the test set
    X_test = all_data[all_data['date_block_num']==i].drop(to_drop_cols, axis = 1)
    y_test = all_data[all_data['date_block_num']==i][['id', 'target']]
    
    # Do prediction on test set
    pred = model.predict(X_test).astype('int')
    print("No. of zeros = " + str(len(pred[pred==0])))
    
    # Add preds to dataset
    sale_tr_val_melt2.loc[sale_tr_val_melt2['date_block_num']==i, 'target'] = pred 

    # Calculate RMSE
    print("MSE = " + str(mean_squared_error(y_test['target'], pred)))
    mse.append(mean_squared_error(y_test['target'], pred))
    
    results['F'+str(i-pred_start+1)] = pred
    
print("mse mean = " + str(np.mean(mse)))
results


day = 1914
No. of zeros = 1854
MSE = 7.497540177107248

day = 1915
No. of zeros = 2015
MSE = 5.829124303050181

day = 1916
No. of zeros = 2102
MSE = 5.4417841915382095

day = 1917
No. of zeros = 2122
MSE = 6.369629386684159

day = 1918
No. of zeros = 2118
MSE = 8.637586093801247

day = 1919
No. of zeros = 2066
MSE = 13.092817317153164

day = 1920
No. of zeros = 2074
MSE = 15.692030173827485

day = 1921
No. of zeros = 2201
MSE = 9.739914726139718

day = 1922
No. of zeros = 2282
MSE = 8.693014102984586

day = 1923
No. of zeros = 2356
MSE = 8.368973433912759

day = 1924
No. of zeros = 2446
MSE = 6.871761233191211

day = 1925
No. of zeros = 2440
MSE = 11.236470974089865

day = 1926
No. of zeros = 2417
MSE = 16.94129222695966

day = 1927
No. of zeros = 2420
MSE = 18.336503771728434

day = 1928
No. of zeros = 2465
MSE = 11.932108888160052

day = 1929
No. of zeros = 2491
MSE = 10.518858642177763

day = 1930
No. of zeros = 2531
MSE = 9.972777959986882

day = 1931
No. of zeros = 2571
MSE = 9.3

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
5829688,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5829689,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5829690,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5829691,HOBBIES_1_004_CA_1_validation,2,1,1,1,1,2,2,1,1,...,0,0,0,0,0,0,0,0,0,0
5829692,HOBBIES_1_005_CA_1_validation,1,1,1,1,1,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
5829693,HOBBIES_1_006_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5829694,HOBBIES_1_007_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5829695,HOBBIES_1_008_CA_1_validation,4,3,4,3,3,3,4,6,5,...,4,5,5,4,3,4,5,4,5,4
5829696,HOBBIES_1_009_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5829697,HOBBIES_1_010_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [253]:
# Store results as csv
results.to_csv("./out/v3_results-" + state + "-" + store + "-" + str(pred_start) + "-" + str(pred_end) + ".csv", index=False)

In [254]:
tocEnd = time.time()
print("Total time = " + str((tocEnd-ticStart)/60.0) + " mins")

Total time = 99.65423995256424 mins


# Findings

Without hyperparam tuning, state='CA', store_id='CA_1':
* If train on 1885, pred on 1886 to 1913, mean mse = 12.61
* If train on <=1885, pred on 1886 to 1913, mean mse = 12.33

# Combine all predictions together and prepare submission file

In [255]:
# # Combine all predictions together
# pred_start = 1914              # do pred starting from this day
# pred_end = 1941                # do pred until and include this day

# state_store_list = [('CA', 'CA_1'), ('CA', 'CA_2'), ('CA', 'CA_3'), ('CA', 'CA_4'),
#                     ('TX', 'TX_1'), ('TX', 'TX_2'), ('TX', 'TX_3'), 
#                     ('WI', 'WI_1'), ('WI', 'WI_2'), ('WI', 'WI_3')]

# state_store = state_store_list[0]
# results_tot = pd.read_csv("./out/results-" + state_store[0] + "-" + state_store[1] + "-" + str(pred_start) + "-" + str(pred_end) + ".csv")


# for state_store in state_store_list[1:]:
#     results = pd.read_csv("./out/v3_results-" + state_store[0] + "-" + state_store[1] + "-" + str(pred_start) + "-" + str(pred_end) + ".csv")
#     results_tot = results_tot.append(results)
    
# results_tot

In [256]:
# # Prepare submission file
# submission = pd.merge(sample_sub[['id']], results_tot, on='id', how='left')
# submission.fillna(value=0, inplace=True)
# submission.to_csv("./out/v3_submission.csv", index=False)
# submission

This obtained weighted RMSSE of ? on the validation set